In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns

## Code which scrapes the recently reviews (only 100 pages - 1,400 reviews)

In [14]:

def ratebeer_scraper(page_start=1, page_end=2, url_set=None):
    
    beer_name = []
    beer_rating = []
    brewery_name = []
    brewery_location = []
    review_content = []
    reviewer_name = []
 
    if url_set == None:
        print("Need url to run")
        return

    for x in tqdm_notebook(range(page_start, page_end + 1)):
        try:
            url_format = url_set.format(x)
            r = requests.get(url_format)
            soup = BeautifulSoup(r.text, 'html.parser')
            table = soup.find('table', attrs={'class': 'table'})
            table_rows = table.find_all('tr')

            for rows in table_rows:

            # Scrape Beer Name    
                try:
                    beer_name.append(rows.find('a', attrs={'style':'font-size:20px; font-weight:bold;'}).text)
                except:
                    beer_name.append(None)
            # Scrape Review Rating  
                try:
                    beer_rating.append(rows.find('span', attrs={'class':'uas'}).text)
                except:
                    beer_rating.append(None)
            # Scrape Brewery Name  
                try:
                    brewery_div = rows.find('div')
                    brewery_name.append(brewery_div.a.text)
                except:
                    brewery_name.append(None)
            # Scrape Brewery Location
                try:
                    brewery_location.append(rows.find('span', attrs={'class':'small'}).text.strip())
                except:
                    brewery_location.append(None)
            # Scrape Reviewer Name
                try:
                    reviewer_name.append(rows.find('span', attrs={'class':'pull-right'}).text.strip())
                except:
                    reviewer_name.append(None)
            # Scrape content
                try:
                    review_content.append(rows.find('div', attrs={'style':"color:#666;"}).text.strip())
                except:
                    review_content.append(None)
        except:
            pass

    beerz = pd.DataFrame({'beer_name': beer_name,
                         'beer_rating': beer_rating,
                        'brewery_name': brewery_name,
                         'brewery_location': brewery_location,
                        'review_content': review_content,
                        'reviewer_name': reviewer_name})
    return beerz



In [23]:
# Scraping most recent reviews
url = 'https://www.ratebeer.com/beer-ratings/0/{}/'
df_temp = ratebeer_scraper(0,100,url)

/Users/lukebetham/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [25]:
# Scraping new beers
url = 'https://www.ratebeer.com/beer-ratings/5/{}/'
df_temp = ratebeer_scraper(0,100,url)

/Users/lukebetham/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [28]:
# Scraping fave beers
url = 'https://www.ratebeer.com/beer-ratings/4/{}/'
df_temp = ratebeer_scraper(0,100,url)

/Users/lukebetham/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [31]:
# Scraping rave beers
url = 'https://www.ratebeer.com/beer-ratings/1/{}/'
df_temp = ratebeer_scraper(0,100,url)

/Users/lukebetham/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
# Scraping rant beers
url = 'https://www.ratebeer.com/beer-ratings/2/{}/'
df_temp = ratebeer_scraper(0,100,url)

In [ ]:
# Scraping anomalous beers
url = 'https://www.ratebeer.com/beer-ratings/2/{}/'
df_temp = ratebeer_scraper(0,100,url)

In [32]:
df_temp

,beer_name,beer_rating,brewery_name,brewery_location,review_content,reviewer_name
0,Prairie Holiday Weekend,4.6,Prairie Artisan Ales,"Tulsa, Oklahoma",Fanatyk Piwa is drinking a Holiday Weekend by ...,Hubson
1,Hidden Springs Black Aggie - Double Barrel Age...,4.8,Hidden Springs Ale Works,"Tampa, Florida",Fanatyk Piwa is drinking a Double Barrel Fluff...,Hubson
2,Angry Chair Tiramisu,4.5,Angry Chair Brewing,"Tampa, Florida",Fanatyk Piwa is drinking a Tiramisu (2019) by ...,Hubson
3,Fort George Matryoshka,4.6,Fort George Brewery,"Astoria, Oregon","500ml bottle, 2020 vintage. Roasted malt aroma...",zoucrew102
4,Dry County Bone Music,4.7,Dry County Brewing Company,"Kennesaw, Georgia",Pours rich coffee with a thick and fairly pers...,Cybercat
...,...,...,...,...,...,...
1510,Pipeworks Baked And Layered,4.5,Pipeworks Brewing Company,"Chicago, Illinois",Pipeworks Brewing Company’s Baked And Layered ...,Sexy_texan18
1511,Bas-Canada Espace Libre,4.5,Brasserie du Bas-Canada,"Gatineau, Quebec","Jaune voilée, nez fruité de melon miel et légè...",QcBeer
1512,O'Fallon Wheach,5.0,O'Fallon Brewery,"Maryland Heights, Missouri",I thought this beer was delicious. It would be...,snider75
1513,Canned Heat Koala,4.5,Canned Heat,"Fall River, Massachusetts","1,900th rating! Pours a great opaque orange ju...",Quick3Beers


In [ ]:
df = pd.concat([df,df_temp])
print(len(df.duplicated()))
df.drop_duplicates(inplace=True)
print(len(df))

UnicodeEncodeError: 'utf-8' codec can't encode characters in position 90-91: surrogates not allowed

In [17]:
# df.to_csv('all_reviews.csv',index=False)
# df = pd.read_csv('all_reviews.csv')